In [82]:
import recognition_helper as helper
import pandas as pd
import numpy as np
import cv2

In [83]:
helper.r.ping()

True

In [84]:
key_to_delete = 'Umesh Khatiwada#2'
deleted_count = helper.r.delete(key_to_delete)
# Check if the key was deleted
if deleted_count == 1:
    print(f"Key '{key_to_delete}' was successfully deleted.")
elif deleted_count == 0:
    print(f"Key '{key_to_delete}' does not exist in the database.")
else:
    print("An error occurred while deleting the key.")

Key 'Umesh Khatiwada#2' does not exist in the database.


In [85]:
## step 1: extract data from database

In [86]:
hash_name = 'school:register'

In [94]:
decoded_dictionary = helper.r.hgetall(name='school:register')
decoded_series = pd.Series(decoded_dictionary)
decoded_series
# Convert each element of retrive_series from bytes to 32-bit float NumPy arrays
decoded_series = decoded_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))

# Retrieve the index of retrive_series
index = decoded_series.index

# Convert the index elements from bytes to strings
index = list(map(lambda x: x.decode(), index))
decoded_series.index = index
decoded_df = decoded_series.to_frame().reset_index()
decoded_df.columns = ['name_role', 'facial_features']
decoded_df.rename(columns={'facial_features':'Facial Features'}, inplace= True)
# Split by '#' and create a new DataFrame with two columns 'Name' and 'Role'
temp_df = decoded_df['name_role'].str.split('#', expand=True)

# Split each part by '_' and capitalize each word
decoded_df[['Name', 'Role']] = temp_df.applymap(lambda x: ' '.join([word.capitalize() for word in x.split('_')]))

decoded_df



,name_role,Facial Features,Name,Role
0,Chris_Evans#student,"[0.7200426, -1.4685223, -0.7779212, -1.460148,...",Chris Evans,Student
1,Morgan_Freeman#teacher,"[0.48528442, -0.619405, -0.4070305, 1.0686811,...",Morgan Freeman,Teacher
2,umesh_khatiwada#student,"[-0.06068934, 1.186499, 1.0322479, 0.8512557, ...",Umesh Khatiwada,Student
3,Scarlett Johansson#student,"[0.19131908, -0.48838148, -1.690325, 0.4423288...",Scarlett johansson,Student
4,Angelina Jolie#student,"[0.43239492, 1.0748208, -0.12874822, -1.833274...",Angelina jolie,Student


In [88]:
## Get Realtime prediction

In [89]:
# define a video capture object 
vid = cv2.VideoCapture(0) 
while(True): 
      
       
    # Capture the video frame by frame 
    ret, frame = vid.read() 
    
    if ret == False:
        print("Unable to read camera")
        break
    
    predicted_frame = helper.name_prediction(frame, decoded_df)
    
    # Display the resulting frame 
#     cv2.imshow('frame', frame) 
     # Display the predict frame 
    cv2.imshow('prediction_frame', predicted_frame) 
    
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) == ord('i'): 
        break
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows()